In [1]:
import tensorflow as tf
import warnings

if not tf.test.gpu_device_name():
    warnings.warn ("mama")
else:
    print(tf.test.gpu_device_name())

print(tf.version.VERSION)


/device:GPU:0
2.1.0


In [2]:
#import sys
#!{sys.executable} -m pip install sk
import os
import torch.nn.functional as F
import torch
import random
import pandas as pd
import keras
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose, Conv3D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [3]:
import numpy
import warnings
from keras.layers import Convolution3D, Input, merge, RepeatVector, Activation
from keras.models import Model
from keras.layers.advanced_activations import PReLU
#from keras import activations, initializations, regularizers
from keras.engine import Layer, InputSpec
#from keras.utils.np_utils import conv_output_length
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from keras.engine.topology import Layer
import functools
import tensorflow as tf
import pickle
import time

In [4]:
X=np.load('D:/aminur/train.npy')
y=np.load('D:/aminur/test.npy')
print(X.shape,type(X))
#X=X.reshape(200,128,128,128)
#print('x',X[1])
print(y.shape,type(y))
#print ('y',y[1])

(200, 128, 128, 128, 1) <class 'numpy.ndarray'>
(200, 128, 128, 128, 1) <class 'numpy.ndarray'>


In [5]:
# Split train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=42)
print(X_train.shape, X_valid.shape,y_train.shape,y_valid.shape)

(180, 128, 128, 128, 1) (20, 128, 128, 128, 1) (180, 128, 128, 128, 1) (20, 128, 128, 128, 1)


In [6]:
from keras import initializers
class Deconvolution3D(Layer):
    def __init__(self, filters, kernel_size, output_shape, subsample):
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = (1,) + subsample + (1,)
        self.output_shape_ = output_shape
        assert K.backend() == 'tensorflow'
        super(Deconvolution3D, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 5
        self.input_shape_ = input_shape
        W_shape = self.kernel_size + (self.filters, input_shape[4],)
        self.W = self.add_weight(shape=W_shape,
                                 initializer=functools.partial(initializers.glorot_uniform()),
                                 name='{}_W'.format(self.name))
        self.b = self.add_weight(shape=(1, 1, 1, self.filters,), initializer='zero', name='{}_b'.format(self.name))
        self.built = True

    def compute_output_shape(self, input_shape):
        return (None,) + self.output_shape_[1:]

    def call(self, x, mask=None):
        return tf.nn.conv3d_transpose(x, self.W, output_shape=self.output_shape_,
                                      strides=self.strides, padding='SAME', name=self.name) + self.b

    def get_config(self):
        base_config = super(Deconvolution3D, self).get_config().copy()
        base_config['output_shape'] = self.output_shape_
        return base_config

In [7]:
from keras import backend as K
from keras.engine import Layer

class Softmax(Layer):
    def __init__(self, axis=-1,**kwargs):
        self.axis=axis
        super(Softmax, self).__init__(**kwargs)

    def build(self,input_shape):
        pass

    def call(self, x,mask=None):
        e = K.exp(x - K.max(x, axis=self.axis, keepdims=True))
        s = K.sum(e, axis=self.axis, keepdims=True)
        return e / s

    def get_output_shape_for(self, input_shape):
        return input_shape

In [8]:
def downward_layer(input_layer, n_convolutions, n_output_channels):
    inl = input_layer

    for _ in range(n_convolutions):
        inl = PReLU()(
            Conv3D(filters=(n_output_channels // 2), kernel_size=5,
                   padding='same', kernel_initializer='he_normal')(inl)
        )
    add_l = add([inl, input_layer])
    downsample = Conv3D(filters=n_output_channels, kernel_size=2, strides=2,
                        padding='same', kernel_initializer='he_normal')(add_l)
    downsample = PReLU()(downsample)
    return downsample, add_l

In [9]:
def upward_layer(input0, input1, n_convolutions, n_output_channels):
    merged = concatenate([input0, input1], axis=4)
    inl = merged
    for _ in range(n_convolutions):
        inl = PReLU()(
            Conv3D((n_output_channels * 4), kernel_size=5,
                   padding='same', kernel_initializer='he_normal')(inl)
        )
    add_l = add([inl, merged])
    shape = add_l.get_shape().as_list()
    new_shape = (1, shape[1] * 2, shape[2] * 2, shape[3] * 2, n_output_channels)
    upsample = Deconvolution3D(n_output_channels, (2, 2, 2), new_shape, subsample=(2, 2, 2))(add_l)
    return PReLU()(upsample)

In [10]:
def vnet(input_size=(128, 128, 128, 1), optimizer=Adam(lr=1e-4),
         loss='binary_crossentropy', metrics=['accuracy']):
         # loss='categorical_crossentropy', metrics=['categorical_accuracy']):
    # Layer 1
    inputs = Input(input_size)
    conv1 = Conv3D(16, kernel_size=5, strides=1, padding='same', kernel_initializer='he_normal')(inputs)
    #conv1 = PReLU()(conv1)
    repeat1 = concatenate(16 * [inputs], axis=-1)
    add1 = add([conv1, repeat1])
    prelu1=PReLU()(add1)
    down1 = Conv3D(32, 2, strides=2, padding='same', kernel_initializer='he_normal')(prelu1)
    down1 = PReLU()(down1)

    # Layer 2,3,4
    down2, add2 = downward_layer(down1, 2, 64)
    down3, add3 = downward_layer(down2, 3, 128)
    down4, add4 = downward_layer(down3, 3, 256)

    # Layer 5
    # !Mudar kernel_size=(5, 5, 5) quando imagem > 64!
    conv_5_1 = Conv3D(256, kernel_size=(5, 5, 5), padding='same', kernel_initializer='he_normal')(down4)
    conv_5_1 = PReLU()(conv_5_1)
    conv_5_2 = Conv3D(256, kernel_size=(5, 5, 5), padding='same', kernel_initializer='he_normal')(conv_5_1)
    conv_5_2 = PReLU()(conv_5_2)
    conv_5_3 = Conv3D(256, kernel_size=(5, 5, 5), padding='same', kernel_initializer='he_normal')(conv_5_2)
    conv_5_3 = PReLU()(conv_5_3)
    add5 = add([conv_5_3, down4])
    aux_shape = add5.get_shape().as_list()
    upsample_5 = Deconvolution3D(128, (2, 2, 2), (1, aux_shape[1]*2,aux_shape[2]*2,
                                                  aux_shape[3]*2, 128), subsample=(2, 2, 2))(add5)
    upsample_5 = PReLU()(upsample_5)

    # Layer 6,7,8
    upsample_6 = upward_layer(upsample_5, add4, 3, 64)
    upsample_7 = upward_layer(upsample_6, add3, 3, 32)
    upsample_8 = upward_layer(upsample_7, add2, 2, 16)

    # Layer 9
    merged_9 = concatenate([upsample_8, add1], axis=4)
    conv_9_1 = Conv3D(32, kernel_size=(5, 5, 5), padding='same', kernel_initializer='he_normal')(merged_9)
    conv_9_1 = PReLU()(conv_9_1)
    add_9 = add([conv_9_1, merged_9])
    # conv_9_2 = Conv3D(1, kernel_size=(1, 1, 1), padding='same', kernel_initializer='he_normal')(add_9)
    conv_9_2 = Conv3D(1, kernel_size=(1, 1, 1), padding='same', kernel_initializer='he_normal')(add_9)
    conv_9_2 = PReLU()(conv_9_2)

    # softmax = Softmax()(conv_9_2)
    sigmoid = Conv3D(1, kernel_size=(1, 1, 1), padding='same', kernel_initializer='he_normal',
                     activation='sigmoid')(conv_9_2)

    model = Model(inputs=inputs, outputs=sigmoid)
    # model = Model(inputs=inputs, outputs=softmax)
    #model.summary()

    #model.compile(optimizer, loss, metrics)

    return model


In [11]:
#model = vnet(input_size=(128, 128, 128, 1))
#model.summary()
save_dir='D:/aminur/vnet/'
weights_dir = save_dir + "weights_vnet.h5"
def dice_coef(y_true, y_pred, smooth, thresh):
    #y_pred =K.cast((K.greater(y_pred,thresh)), dtype='float32')#转换为float型
    #y_pred = y_pred[y_pred > thresh]=1.0
    y_true_f =y_true# K.flatten(y_true)
    y_pred_f =y_pred# K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f,axis=(0,1,2))
    denom =K.sum(y_true_f,axis=(0,1,2)) + K.sum(y_pred_f,axis=(0,1,2))
    return K.mean((2. * intersection + smooth) /(denom + smooth))

def dice_loss(smooth, thresh):
    def dice(y_true, y_pred):
        
        return 1-dice_coef(y_true, y_pred, smooth, thresh)
    return dice
model_dice=dice_loss(smooth=1e-5,thresh=0.5)
model=vnet(input_size = (128,128,128,1))
#model_dice=dice_loss(smooth=1e-5,thresh=0.5)
opt=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
model.compile(optimizer=opt, loss=model_dice, metrics=['accuracy','mse'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 128 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 128, 128, 128 2016        input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 128, 128, 128 0           input_1[0][0]                    
                                                                 input_1[0][0]                    
                                                                 input_1[0][0]                    
                                                                 input_1[0][0]              

In [12]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint(weights_dir, monitor='val_loss',mode='auto',verbose=1, save_best_only=True, save_weights_only=True)]
results = model.fit(X_train, y_train, batch_size=1, epochs=4,callbacks=callbacks,
                    validation_data=(X_valid, y_valid),verbose=1)

Train on 180 samples, validate on 20 samples
Epoch 1/4


KeyboardInterrupt: 